# tenures

## Import and cleanup

Let's start with importing important modules and loading the `tenures` table.

In [82]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from math import nan
import re

# import from CSV
tenures = pd.read_csv("presidents-of-chile_tenures.csv")

# quick view of what the table contains
print('---------- Quick view of the tenures dataset ----------')
print(tenures.head(3))
print('\n')
print(tenures.tail(3))


---------- Quick view of the tenures dataset ----------
  tenure_id person_id first_surname second_surname given_name  \
0     T0001     P0001        Blanco       Encalada     Manuel   
1     T0002     P0002    Eyzaguirre     Arechavala    Agustín   
2     T0003     P0003        Freire        Serrano      Ramón   

  date_elected_appointed  years_tenure  is_elected  is_interim  is_military  \
0             1826-07-08           NaN           1           1            0   
1             1826-09-09           NaN           0           1            0   
2             1827-01-25           NaN           0           1            0   

   is_junta  vacant_presidency date_tenure_start date_tenure_end  is_resigned  \
0         0                  0        1826-07-09      1826-09-09          1.0   
1         0                  0        1826-09-09      1827-01-25          1.0   
2         0                  0        1827-01-25      1827-02-15          0.0   

   is_deposed  
0         0.0  
1        

In [83]:
# quick view of what columns we have
print(tenures.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   tenure_id               81 non-null     object 
 1   person_id               79 non-null     object 
 2   first_surname           79 non-null     object 
 3   second_surname          79 non-null     object 
 4   given_name              79 non-null     object 
 5   date_elected_appointed  79 non-null     object 
 6   years_tenure            41 non-null     float64
 7   is_elected              81 non-null     int64  
 8   is_interim              81 non-null     int64  
 9   is_military             81 non-null     int64  
 10  is_junta                81 non-null     int64  
 11  vacant_presidency       81 non-null     int64  
 12  date_tenure_start       81 non-null     object 
 13  date_tenure_end         80 non-null     object 
 14  is_resigned             78 non-null     floa

We need dates to be formatted as dates, so let's convert them below.

In [84]:
# format columns as dates
tenures['date_elected_appointed'] = pd.to_datetime(tenures['date_elected_appointed'], format='%Y-%m-%d')
tenures['date_tenure_start'] = pd.to_datetime(tenures['date_tenure_start'], format='%Y-%m-%d')
tenures['date_tenure_end'] = pd.to_datetime(tenures['date_tenure_end'], format='%Y-%m-%d')

# quick view of what columns we have
print(tenures.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   tenure_id               81 non-null     object        
 1   person_id               79 non-null     object        
 2   first_surname           79 non-null     object        
 3   second_surname          79 non-null     object        
 4   given_name              79 non-null     object        
 5   date_elected_appointed  79 non-null     datetime64[ns]
 6   years_tenure            41 non-null     float64       
 7   is_elected              81 non-null     int64         
 8   is_interim              81 non-null     int64         
 9   is_military             81 non-null     int64         
 10  is_junta                81 non-null     int64         
 11  vacant_presidency       81 non-null     int64         
 12  date_tenure_start       81 non-null     datetime64[n

We'll also try to convert  `is_resigned` and `is_deposed` into int64, like the other int64 data points with zeroes and ones.

In [85]:
# format columns as integers
tenures['is_resigned'] = tenures['is_resigned'].convert_dtypes(convert_integer=True)
tenures['is_deposed'] = tenures['is_deposed'].convert_dtypes(convert_integer=True)

# quick view of what columns we have
print(tenures.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   tenure_id               81 non-null     object        
 1   person_id               79 non-null     object        
 2   first_surname           79 non-null     object        
 3   second_surname          79 non-null     object        
 4   given_name              79 non-null     object        
 5   date_elected_appointed  79 non-null     datetime64[ns]
 6   years_tenure            41 non-null     float64       
 7   is_elected              81 non-null     int64         
 8   is_interim              81 non-null     int64         
 9   is_military             81 non-null     int64         
 10  is_junta                81 non-null     int64         
 11  vacant_presidency       81 non-null     int64         
 12  date_tenure_start       81 non-null     datetime64[n

Let's create a new column called `years_actual_tenure` which is the number of years between the `date_tenure_end` and `date_tenure_start`.

In [86]:
# get date today and format it as YYYY-MM-DD
date_today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'), format='%Y-%m-%d')

# for the current president whose date_tenure_end is null, fill date_tenure_end with the current date
for index, value in tenures['date_tenure_end'].items():
    if pd.isnull(value):
        tenures.at[index, 'date_tenure_end'] = date_today

# do the calculating of years_actual_tenure
tenures['years_actual_tenure'] = abs(tenures['date_tenure_end'] - tenures['date_tenure_start'])

for index, value in tenures['years_actual_tenure'].items():
    # pandas timedelta is different from native python timedelta, so need to convert it
    value.to_pytimedelta()
    # force the float 365.25 into a timedelta so that the next division returns a float
    days_in_a_yr = timedelta(days=365.25) 
    # divide by 365.25 to convert days to years, then assign year value to 'years_actual_tenure' column
    value = value / days_in_a_yr
    tenures.at[index,'years_actual_tenure'] = float(value)

tenures['years_actual_tenure'].convert_dtypes(convert_floating=True)

# quick view of what the table contains
print('---------- Quick view of the tenures dataset ----------')
print(tenures.head(3))
print('\n')
print(tenures.tail(3))
print('\n')

# quick view of what columns we have
print(tenures.info())


---------- Quick view of the tenures dataset ----------
  tenure_id person_id first_surname second_surname given_name  \
0     T0001     P0001        Blanco       Encalada     Manuel   
1     T0002     P0002    Eyzaguirre     Arechavala    Agustín   
2     T0003     P0003        Freire        Serrano      Ramón   

  date_elected_appointed  years_tenure  is_elected  is_interim  is_military  \
0             1826-07-08           NaN           1           1            0   
1             1826-09-09           NaN           0           1            0   
2             1827-01-25           NaN           0           1            0   

   is_junta  vacant_presidency date_tenure_start date_tenure_end  is_resigned  \
0         0                  0        1826-07-09      1826-09-09            1   
1         0                  0        1826-09-09      1827-01-25            1   
2         0                  0        1827-01-25      1827-02-15            0   

   is_deposed years_actual_tenure  
0    

## Presidents who did not complete their tenure

Investigate: Which presidents did not complete the term they were elected for, and why?

In [87]:
# create new DF with non-null years_tenure
incomplete_tenures = tenures[tenures['years_tenure'] > 0]

# create tenure_diff and completed_term to check if someone didn't compelte their tenure

## this function calculates difference between the expected tenure and the actual tenure
def tenure_diff():
    # below line is equivalent to - tenures.loc[:, 'tenure_diff'] = abs(tenures['years_tenure'] - tenures['years_actual_tenure'])
    incomplete_tenures['tenure_diff'[:]] = abs(incomplete_tenures['years_tenure'] - incomplete_tenures['years_actual_tenure'])

tenure_diff()

## below line is equivalent to - tenures.loc[:, 'completed_term'] = ''
incomplete_tenures['completed_term'[:]] = ''

## this function calculates whether someone completed their term based on tenure_diff
def completed_term():
    for index, value in incomplete_tenures.iterrows():
        # if the difference is negligible then the tenure is completed
        if round(value['tenure_diff'], 2) <= 0.01:
            incomplete_tenures.at[index, 'completed_term'] = 1
        # exclude the current president
        elif value['date_tenure_end'] == date_today:
            incomplete_tenures.at[index, 'completed_term'] = nan
        else:
            incomplete_tenures.at[index, 'completed_term'] = 0

completed_term()

# create new DF with all presidents who didn't complete their terms
incomplete_tenures = incomplete_tenures[incomplete_tenures['completed_term'] == 0]
print('---------- Presidents of Chile who did not complete their tenure ----------')
print(incomplete_tenures)

# DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array.
# To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
# persons.loc[:, 'age'] = abs(persons['date_of_death'] - persons['date_of_birth'])


---------- Presidents of Chile who did not complete their tenure ----------
   tenure_id person_id first_surname second_surname    given_name  \
26     T0025     P0016     Balmaceda      Fernández   José Manuel   
29     T0028     P0018         Montt        Álvarez         Jorge   
30     T0029     P0019     Errázuriz      Echaurren      Federico   
33     T0032     P0022         Montt          Montt         Pedro   
38     T0037     P0027    Alessandri          Palma        Arturo   
42     T0036     P0027    Alessandri          Palma        Arturo   
44     T0042     P0024      Figueroa        Larraín      Emiliano   
46     T0044     P0032        Ibáñez      del Campo        Carlos   
51     T0048     P0034       Montero      Rodríguez  Juan Esteban   
57     T0054     P0040       Aguirre          Cerda         Pedro   
59     T0056     P0042          Ríos        Morales  Juan Antonio   
68     T0064     P0049       Allende        Gossens      Salvador   

   date_elected_appointed 

C:\Users\canogle\AppData\Local\Temp\1\ipykernel_30944\1178208902.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_tenures['tenure_diff'[:]] = abs(incomplete_tenures['years_tenure'] - incomplete_tenures['years_actual_tenure'])
C:\Users\canogle\AppData\Local\Temp\1\ipykernel_30944\1178208902.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_tenures['completed_term'[:]] = ''


Jorge Montt was elected for a term from 1891-12-26 to 1896-09-18, which isn't a whole 5 years. Let's fix this so that Jorge Montt doesn't appear as if he didn't complete the term he was elected for.

In [88]:
# get exact number of years_tenure for Jorge Montt
jorge_montt_tenure = (date(1896, 9, 18) - date(1891, 12, 26)) / days_in_a_yr
incomplete_tenures.at[29, 'years_tenure'] = float(jorge_montt_tenure)
incomplete_tenures['years_tenure'].convert_dtypes(convert_floating=True)

print(f'Jorge Montt was actually elected for a tenure of {jorge_montt_tenure} years, and he was able to complete this tenure.')
print('\n')
print(incomplete_tenures.loc[29, :])
print('\n')

# since we updated Jorge Montt's year_tenure, we have to recalculate tenure_diff and completed_term
tenure_diff()
completed_term()

# check if we don't have Jorge Montt anymore
incomplete_tenures = incomplete_tenures[incomplete_tenures['completed_term'] == 0]
print('---------- Presidents of Chile who did not complete their tenure ----------')
print(incomplete_tenures)


Jorge Montt was actually elected for a tenure of 4.731006160164271 years, and he was able to complete this tenure.


tenure_id                               T0028
person_id                               P0018
first_surname                           Montt
second_surname                        Álvarez
given_name                              Jorge
date_elected_appointed    1891-10-18 00:00:00
years_tenure                         4.731006
is_elected                                  1
is_interim                                  0
is_military                                 0
is_junta                                    0
vacant_presidency                           0
date_tenure_start         1891-12-26 00:00:00
date_tenure_end           1896-09-18 00:00:00
is_resigned                                 0
is_deposed                                  0
years_actual_tenure                  4.731006
tenure_diff                          0.268994
completed_term                              0
Name: 29,

Why didn't these people complete their terms? Let's filter `imcomplete_tenures` so that we only have the relevant columns for investigation.

In [89]:
# filter only relevant columns for incomplete_tenures
incomplete_tenures = incomplete_tenures.loc[:, ['tenure_id', 'person_id', 'first_surname', 'second_surname', 'given_name',
                                            'date_tenure_start', 'date_tenure_end', 'years_tenure', 'years_actual_tenure', 'is_resigned', 'is_deposed']]
print('---------- Presidents of Chile who did not complete their tenure ----------')
print(incomplete_tenures)

---------- Presidents of Chile who did not complete their tenure ----------
   tenure_id person_id first_surname second_surname    given_name  \
26     T0025     P0016     Balmaceda      Fernández   José Manuel   
30     T0029     P0019     Errázuriz      Echaurren      Federico   
33     T0032     P0022         Montt          Montt         Pedro   
38     T0037     P0027    Alessandri          Palma        Arturo   
42     T0036     P0027    Alessandri          Palma        Arturo   
44     T0042     P0024      Figueroa        Larraín      Emiliano   
46     T0044     P0032        Ibáñez      del Campo        Carlos   
51     T0048     P0034       Montero      Rodríguez  Juan Esteban   
57     T0054     P0040       Aguirre          Cerda         Pedro   
59     T0056     P0042          Ríos        Morales  Juan Antonio   
68     T0064     P0049       Allende        Gossens      Salvador   

   date_tenure_start date_tenure_end  years_tenure years_actual_tenure  \
26        1886-09-18 

Several reasons why presidents didn't complete their terms: 
* They resigned
* They were deposed in a coup or a civil war
* They died in office
* Or a combination of these

We have `date_of_death` in the `persons` table so that we can check whether someone died in office. This requires importing the `persons` table and joining it with the `incomplete_tenures` table.

Here we'll import and clean the `persons` table first. Then we'll join `persons` into `incomplete_tenures` and create `is_died_in_office`.

Extra: let's calculate their age upon leaving office (`age_left_office`), which is more interesting for those who died in office.

In [90]:
# import persons table
persons = pd.read_csv('presidents-of-chile_persons.csv')

# find and clean up wrongly formatted dates by making them null
print('---------- These persons have incomplete dates of birth and death ----------')
print(persons[persons['date_of_birth'].str.contains('00-?00', regex=True) == True])  # index 6, 22, 35
print(persons[persons['date_of_death'].str.contains('00-?00', regex=True) == True])  # index 35
print("\n")

persons.loc[[6, 22, 35], 'date_of_birth'] = ''
persons.loc[35, 'date_of_death'] = ''

# convert dates to date format
persons['date_of_birth'] = pd.to_datetime(persons['date_of_birth'], format='%Y-%m-%d')
persons['date_of_death'] = pd.to_datetime(persons['date_of_death'], format='%Y-%m-%d')

# quick view of what the table contains
# print('---------- Quick view of the persons dataset ----------')
# print(persons.head(3))
# print("\n")
# print(persons.tail(3))
# print("\n")
# print(persons.info())

# set index on both tables so that the joining works (hopefully)
persons.set_index('person_id')
incomplete_tenures.set_index('person_id')

# join on person_id
incomplete_tenures = incomplete_tenures.merge(persons, on='person_id')
incomplete_tenures = incomplete_tenures.loc[:, ['tenure_id', 'person_id', 'first_surname', 'second_surname', 'given_name',
                                                'date_tenure_start', 'date_tenure_end', 'years_tenure', 'years_actual_tenure',
                                                'date_of_birth', 'date_of_death', 'is_resigned', 'is_deposed']]

# create column is_died_in_office - if the end of their tenure is the same as their date of death, then they died in office
incomplete_tenures['is_died_in_office'[:]] = incomplete_tenures['date_tenure_end'].eq(incomplete_tenures['date_of_death'])

for index, value in incomplete_tenures['is_died_in_office'].items():
    if value == True:
        incomplete_tenures.at[index, 'is_died_in_office'] = 1
    else:
        incomplete_tenures.at[index, 'is_died_in_office'] = 0

# create column age_left_office
incomplete_tenures['age_left_office'[:]] = abs(incomplete_tenures['date_tenure_end'] - incomplete_tenures['date_of_birth'])

for index, value in incomplete_tenures['age_left_office'].items():
    value.to_pytimedelta()
    value = value / days_in_a_yr
    incomplete_tenures.at[index, 'age_left_office'] = float(value)

# print only the important columns
incomplete_tenures = incomplete_tenures.loc[:, ['tenure_id', 'person_id', 'first_surname', 'second_surname', 'given_name',
                                                'date_tenure_start', 'date_tenure_end', 'years_tenure', 'years_actual_tenure',
                                                'age_left_office', 'date_of_birth', 'date_of_death', 'is_resigned', 'is_deposed',
                                                'is_died_in_office']]
print('---------- Presidents of Chile who did not complete their tenure ----------')
print(incomplete_tenures)


---------- These persons have incomplete dates of birth and death ----------
   person_id                                          full_name date_of_birth  \
6      P0007  Francisco Antonio Pascual de la Ascensión Ruiz...    1790-00-00   
22     P0023                             Elías Fernández Albano    1845-00-00   
35     P0036                                 Arturo Puga Osorio    1879-00-00   

   date_of_death place_of_birth        region_of_birth  is_woman  
6     1860-03-23       Santiago  Santiago Metropolitan         0  
22    1910-09-06       Santiago  Santiago Metropolitan         0  
35    0000-04-28       Santiago  Santiago Metropolitan         0  
   person_id           full_name date_of_birth date_of_death place_of_birth  \
35     P0036  Arturo Puga Osorio    1879-00-00    0000-04-28       Santiago   

          region_of_birth  is_woman  
35  Santiago Metropolitan         0  


---------- Presidents of Chile who did not complete their tenure ----------
   tenure_id pers

This is where it gets pretty morbid: some presidents died due to illness, while some died by suicide. For that, we have the `deaths` table which we will join with `incomplete_tenures`.

In [91]:
# import deaths table
deaths = pd.read_csv('presidents-of-chile_deaths.csv')

# convert floats to ints
deaths['is_natural_causes'] = deaths['is_natural_causes'].convert_dtypes(convert_integer=True)
deaths['is_suicide'] = deaths['is_suicide'].convert_dtypes(convert_integer=True)
deaths['is_killed'] = deaths['is_killed'].convert_dtypes(convert_integer=True)
deaths['is_accident'] = deaths['is_accident'].convert_dtypes(convert_integer=True)
deaths['is_disputed'] = deaths['is_disputed'].convert_dtypes(convert_integer=True)
deaths['is_unknown'] = deaths['is_unknown'].convert_dtypes(convert_integer=True)

# quick view of what the table contains
# print('---------- Quick view of the deaths dataset ----------')
# print(deaths.tail(10))
# print("\n")
# print(deaths.info())

# set index on both tables so that the joining works (hopefully)
deaths.set_index('person_id')

# join on person_id
incomplete_tenures = incomplete_tenures.merge(deaths, on='person_id')

# create cause_of_death column
incomplete_tenures['cause_of_death'[:]] = ''

for index, value in incomplete_tenures.iterrows():
    if value['is_unknown'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'unknown'
    elif value['is_disputed'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'disputed'
    elif value['is_natural_causes'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'natural causes'
    elif value['is_suicide'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'suicide'
    elif value['is_killed'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'murder'
    elif value['is_accident'] == 1:
        incomplete_tenures.at[index, 'cause_of_death'] = 'accident'

# print only the important columns
incomplete_tenures = incomplete_tenures.loc[:, ['tenure_id', 'person_id', 'first_surname', 'second_surname', 'given_name',
                                                'date_tenure_start', 'date_tenure_end', 'years_tenure', 'years_actual_tenure',
                                                'age_left_office', 'date_of_birth', 'date_of_death', 'is_resigned', 'is_deposed',
                                                'is_died_in_office', 'cause_of_death']]
print('---------- Presidents of Chile who did not complete their tenure ----------')
print(incomplete_tenures)


---------- Presidents of Chile who did not complete their tenure ----------
   tenure_id person_id first_surname second_surname    given_name  \
0      T0025     P0016     Balmaceda      Fernández   José Manuel   
1      T0029     P0019     Errázuriz      Echaurren      Federico   
2      T0032     P0022         Montt          Montt         Pedro   
3      T0037     P0027    Alessandri          Palma        Arturo   
4      T0036     P0027    Alessandri          Palma        Arturo   
5      T0042     P0024      Figueroa        Larraín      Emiliano   
6      T0044     P0032        Ibáñez      del Campo        Carlos   
7      T0048     P0034       Montero      Rodríguez  Juan Esteban   
8      T0054     P0040       Aguirre          Cerda         Pedro   
9      T0056     P0042          Ríos        Morales  Juan Antonio   
10     T0064     P0049       Allende        Gossens      Salvador   

   date_tenure_start date_tenure_end  years_tenure years_actual_tenure  \
0         1886-09-18 